In [27]:
!pip install geopy
from sklearn import preprocessing
import numpy as np
import pandas as pd
import glob
import geopy
import geopy.distance
import datetime
import sys
import time
#Training part of the data

from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt

!pip install hyperopt
from hyperopt import fmin, tpe, hp,STATUS_OK, Trials

!pip install tpot
!pip install xgboost
#pip install IProgress
#!pip install ipywidgets
!pip install --upgrade jupyter_client
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#Tuning using genetic algorithim using TPOT library. install using ' pip install tpot ''

from tpot import TPOTRegressor
#from ipywidgets import IntProgres

Requirement already up-to-date: jupyter_client in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (6.1.7)
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [37]:
# #only works with google hosted runtime 
# from google.colab import drive
# drive.flush_and_unmount()
# drive.mount('/content/drive')

# Regression Training Experiment

For the regression training , we will be using the 'final_ship_data' file which contains the preprocessed data of ships coming from both the Northern and Southern Routes.

In [28]:
#path = r"/content/drive/My Drive/ship_data_files /final_ship_data.csv"    # for hosted runtime use this path 

path = r"final_ship_data.csv" #for local runtime

                #remove lat2 ,lon2 
                
df=pd.read_csv(path,sep=',',low_memory=False)
test_data=df
df

,Length,Breadth,Draught,Longitude,Latitude,SOG,ETA,Distance remaining,speed
0,152.0,24.0,6.8,5.838777,53.63944,14.3,12.685507,440.630956,34.734991
1,152.0,24.0,6.8,5.845800,53.64076,14.3,12.667177,439.857752,34.724213
2,152.0,24.0,6.8,5.852149,53.64198,14.3,12.650496,439.158871,34.714754
3,152.0,24.0,6.8,5.858515,53.64320,14.3,12.633836,438.458162,34.705070
4,152.0,24.0,6.8,5.864876,53.64441,14.3,12.617179,437.758031,34.695397
...,...,...,...,...,...,...,...,...,...
186520,229.0,38.0,13.3,9.806900,53.55355,7.4,0.083519,1.808408,21.652544
186521,229.0,38.0,13.3,9.810224,53.55293,7.3,0.066774,1.434516,21.483189
186522,229.0,38.0,13.3,9.813442,53.55230,7.2,0.049974,1.071975,21.450709
186523,229.0,38.0,13.3,9.816650,53.55172,7.1,0.033443,0.711492,21.275079


## Data preperation (Train-test split)
The data set is now split into the training and test set. The training set will have 75% of the original data set,while the test data has 25% of the data set. The features of the data set are: 
<ol>
<li>Length</li>
<li>Breadth</li>
<li>Draught</li>
<li>Longitude</li>
<li>Latitude</li>
<li>SOG</li>
<li>Distance remianing</li>
<li>Speed</li>

</ol>
The labels is the estimated time of arrival(ETA)
For the missing values of the features,the imputer function is used to infer missing values of the data the strategy used here is the 'mean

In [29]:
train=test_data

#Features
X_Train = train.drop('ETA', axis=1)

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_Train)


# Impute our data, then train
X_train_imp = imp.transform(X_Train)
X= np.array(X_train_imp)

# Labels
Y= np.array(train['ETA'])

min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X)
X=X_train_minmax
Y.size

186525

In [30]:
X

array([[8.45771144e-02, 1.50684932e-01, 2.90598291e-01, ...,
        5.89958159e-01, 8.85479736e-01, 2.87810730e-01],
       [8.45771144e-02, 1.50684932e-01, 2.90598291e-01, ...,
        5.89958159e-01, 8.83925305e-01, 2.87694706e-01],
       [8.45771144e-02, 1.50684932e-01, 2.90598291e-01, ...,
        5.89958159e-01, 8.82520293e-01, 2.87592876e-01],
       ...,
       [2.12271973e-01, 3.42465753e-01, 8.46153846e-01, ...,
        2.92887029e-01, 1.80171753e-03, 1.44800810e-01],
       [2.12271973e-01, 3.42465753e-01, 8.46153846e-01, ...,
        2.88702929e-01, 1.07701073e-03, 1.42910086e-01],
       [2.12271973e-01, 3.42465753e-01, 8.46153846e-01, ...,
        2.88702929e-01, 3.63142472e-04, 1.44082390e-01]])

In [31]:
#Split train and test
train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size = 0.25, random_state = 5)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (139893, 8)
Training Labels Shape: (139893,)
Testing Features Shape: (46632, 8)
Testing Labels Shape: (46632,)


In [32]:
N_ESTIMATOR =10

# Random forest trainnig (No tuning)
For the training, Random forest regression was selected for the experiment. In this section ,the regression is train with their default values(no tuning) is used to train and to predict with the test data.


In [33]:
#Training of the model, for this example using Random forest regressor

start_time = time.time()
rf = RandomForestRegressor(verbose=10,n_estimators=N_ESTIMATOR,n_jobs=-1)

start_time = time.time()

rf.fit(train_features, train_labels)
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

time_noHpo= time.time() - start_time
test_features

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    2.0s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    2.1s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.1s finished


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


array([[0.17578773, 0.23287671, 0.2991453 , ..., 0.49790795, 0.02440506,
        0.29220158],
       [0.20563847, 0.24657534, 0.57264957, ..., 0.45188285, 0.12725067,
        0.1717478 ],
       [0.41293532, 0.45205479, 0.76068376, ..., 0.23849372, 0.43586908,
        0.27044907],
       ...,
       [0.29187396, 0.36986301, 0.60683761, ..., 0.79497908, 0.63529254,
        0.00296428],
       [0.13598673, 0.26027397, 0.44444444, ..., 0.43096234, 0.08142411,
        0.1362038 ],
       [0.13598673, 0.26027397, 0.52991453, ..., 0.46861925, 0.59828468,
        0.04423233]])

In [34]:
test_labels

array([ 0.35043611,  2.64979611,  6.55105444, ..., 38.20745083,
        1.96966306, 24.59149167])

In [35]:
mean1 = np.sqrt(mean_squared_error(predictions, test_labels))

print("%s seconds it took to train " % (time_noHpo))
print('\nRoot mean squared of Model without training', mean1)

3.125159502029419 seconds it took to train 

Root mean squared of Model without training 0.04898058969128082


# Tuning Section

This section now deals with tuining the Logistic regression

In [36]:
# Use the random grid to search for best hyperparameters

rfh = RandomForestRegressor()
rfh.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Tuning Section
The following HPO algorithims will be used for tuning
1.Random search
2.Bayesian Optimization
3.Genetic Algorithims

For the logistic regression algorithm. The following hyperparameters are used for tuning
- Max_depth : Maximum depth of the trees.
- Maximum features: The maximum number of features to consider when looking for best split.
- Minimum Samples leaf :Minimum samples required for it to be a leaf node.
- Minimum Samples split :The minimum number of samples for the regressor to split a node.



## Random Search
With the hyperparameters mentioned above , a 'search space' is used to sample from and find the best possible hyperparameters for logistic regresssion.

#Now to set the values needed for the parameters to do the random search, make combinations for the different parameters
#important HPO max features leave as auto, sample size> Low no.  Node size> Low values
max_depth=[1,32]
max_features = ['auto']
min_samples_leaf = [ 2,3]
min_samples_split = [ 5]
n_estimators = [int(x) for x in np.linspace(start = 1, stop =N_ESTIMATOR , num = 10)]
bootstrap=[True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

In [38]:
#Training of models with the tuning parameters made
start_time = time.time()


rf_random = RandomizedSearchCV(estimator = rfh, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=5, n_jobs =-1)
rf_random.fit(train_features, train_labels)

time_HpoRandom= time.time() - start_time

Fitting 3 folds for each of 80 candidates, totalling 240 fits


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 80 is smaller than n_iter=100. Running 80 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   29.0s
[Parallel(n_jo

In [39]:
rf_random.best_params_

{'n_estimators': 10,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 32,
 'bootstrap': True}

In [40]:
#Testing on the test features, predictions for random search
predictions2=rf_random.predict(test_features)


In [41]:
predictions2

array([ 0.34888206,  2.64317915,  6.54849196, ..., 38.20134175,
        1.9608904 , 24.61175018])

In [42]:

mean2 =np.sqrt(mean_squared_error(predictions2, test_labels))
print("%s seconds it took to train " % (time_HpoRandom))
print('\nRoot mean squared of Model with Random search', mean2)

148.09545183181763 seconds it took to train 

Root mean squared of Model with Random search 0.05337023816473125


# Bayesian Optimizaition

With the same search space defined in the random search, the Bayesian optimization is used to train the Random forest regressor. The number of trials for the bayesian optimization will be 100 trials.

In [44]:
#Tuning with Bayesian optimization

#Define search space
# min leaf> Low values , min split> low values  ,  n_estimators > Higher number of trees
start_time = time.time()

search_space= {'max_depth':hp.choice('max_depth',range(1,32)),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0.1, 1.0),
        'n_estimators' : hp.choice('n_estimators', (1,N_ESTIMATOR))
    }

def objective(search_space):
    bayes = RandomForestRegressor(
                                 max_depth=search_space['max_depth'],
                                 max_features = search_space['max_features'],
                                 min_samples_leaf = search_space['min_samples_leaf'],
                                 min_samples_split = search_space['min_samples_split'],
                                 n_estimators = search_space['n_estimators'], 
                               )
    accuracy = cross_val_score(bayes, train_features, train_labels, cv =100, n_jobs= -1).mean()
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()
best = fmin(fn= objective,
            space= search_space,
            algo= tpe.suggest,
            max_evals = 100,
            trials= trials)
best

# Training from bayes tuning
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}

trainedforest = RandomForestRegressor( 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = best['n_estimators']
                                      ).fit(train_features, train_labels)
time_HpoBaysian= time.time() - start_time
predictions3= trainedforest.predict(test_features)

100%|██████████| 100/100 [28:51<00:00, 17.31s/trial, best loss: -0.9073947748151242]


In [45]:
predictions3

array([ 0.91861778,  2.72899828,  6.60458063, ..., 34.85679487,
        0.91861778, 34.85679487])

In [46]:
mean3 = np.sqrt(mean_squared_error(predictions3, test_labels))
print("%s seconds it took to train " % (time_HpoBaysian))
print('\nRoot mean squared of Model with Bayesian', mean3)

1731.6784174442291 seconds it took to train 

Root mean squared of Model with Bayesian 3.3906692822837488


In [47]:
best

{'max_depth': 15,
 'max_features': 0,
 'min_samples_leaf': 0.0018407207451178742,
 'min_samples_split': 0.1038902953428581,
 'n_estimators': 1}

## Genetic Algrorithim

For the Genetic Algorithim 15 Generations are used and evaluated based on the mean squared error. The maximum evaluation time is 5 minutes

In [48]:
start_time = time.time()
               
# Current values for object TPOTRegressor. Constructor has initial 100 generations

tpot_regressor= TPOTRegressor(generations=15, population_size=30,
                         offspring_size=12, mutation_rate=0.9,
                         crossover_rate=0.1,
                         scoring='neg_mean_squared_error', cv=5,
                         subsample=1.0, n_jobs=-1,
                         max_time_mins=None, max_eval_time_mins=5,
                         random_state=None, config_dict=None,
                         template=None,
                         warm_start=False,
                         memory=None,
                         use_dask=False,
                         periodic_checkpoint_folder=None,
                         early_stop=None,
                         verbosity=3,
                         disable_update_check=False)



tpot_regressor.fit(train_features, train_labels) 

30 operators have been imported by TPOT.


Version 0.11.5 of tpot is outdated. Version 0.11.6 was released 2 days ago.


Skipped pipeline #18 due to time out. Continuing to the next pipeline.
Skipped pipeline #33 due to time out. Continuing to the next pipeline.
Skipped pipeline #43 due to time out. Continuing to the next pipeline.

Generation 1 - Current Pareto front scores:
_pre_test decorator: _random_mutation_operator: num_test=0 No feature in X meets the variance threshold 0.20000.stRegressor__max_features=0.5, RandomForestRegressor__min_samples_leaf=8, RandomForestRegressor__min_samples_split=2, RandomForestRegressor__n_estimators=100)

Generation 2 - Current Pareto front scores:
Skipped pipeline #58 due to time out. Continuing to the next pipeline.Regressor__n_neighbors=1, KNeighborsRegressor__p=1, KNeighborsRegressor__weights=uniform)

Generation 3 - Current Pareto front scores:
-1	-0.0025021245515788864	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=1, KNeighborsRegressor__p=1, KNeighborsRegressor__weights=uniform)
Generation 4 - Current Pareto front scores:
_pre_test decorat

TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=15,
              log_file=<ipykernel.iostream.OutStream object at 0x7f0558357a58>,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=-1, offspring_size=12,
              periodic_checkpoint_folder=None, population_size=30,
              random_state=None, scoring='neg_mean_squared_error',
              subsample=1.0, template=None, use_dask=False, verbosity=3,
              warm_start=False)

In [50]:
time_HpoTpot= time.time() - start_time
predictionsTpot = tpot_regressor.predict(test_features)

In [51]:
predictionsTpot

array([ 0.33377583,  2.63334611,  6.53474333, ..., 38.2241575 ,
        1.98626667, 24.60817583])

In [52]:
mean4 =np.sqrt(mean_squared_error(predictionsTpot,test_labels))
print("%s seconds it took to train " % (time_HpoTpot))
print('Root mean squared of Model with Genetic algorithim', mean4)

8942.04030418396 seconds it took to train 
Root mean squared of Model with Genetic algorithim 0.02433704331682046


# Results

The results for Random forest regression (Root mean squared error in hours). For the time it took to tune is displayed on each section of the HPO algorithims tuning sections.

In [53]:
print('Root mean squared of Model without training', mean1)
print('Root mean squared of Model with Random search', mean2)
print('Root mean squared of Model with Bayesian', mean3)
print('Root mean squared of Model with Genetic algorithim', mean4)

Root mean squared of Model without training 0.04898058969128082
Root mean squared of Model with Random search 0.05337023816473125
Root mean squared of Model with Bayesian 3.3906692822837488
Root mean squared of Model with Genetic algorithim 0.02433704331682046
